In [1]:
# migrate_data.py
import os
from datetime import datetime
from supabase import create_client
from pymongo import MongoClient
from dotenv import load_dotenv
import time

load_dotenv()

class DataMigrator:
    def __init__(self):
        # MongoDB Connection
        mongodb_uri = os.getenv("MONGODB_URI")
        self.mongo_client = MongoClient(mongodb_uri)
        self.mongo_db = self.mongo_client[os.getenv("MONGODB_DATABASE", "ats_system")]
        
        # Supabase Connection
        supabase_url = os.getenv("SUPABASE_URL")
        supabase_key = os.getenv("SUPABASE_ANON_KEY")
        self.supabase = create_client(supabase_url, supabase_key)
        
        print("🚀 Data Migration Tool Initialized")
    
    def migrate_all_data(self):
        """Migrate all data from Supabase to MongoDB"""
        print("\n" + "="*60)
        print("📦 STARTING DATA MIGRATION: Supabase → MongoDB Atlas")
        print("="*60)
        
        # Migration sequence
        migrations = [
            self.migrate_candidates,
            self.migrate_jobs,
            self.migrate_applications,
        ]
        
        total_migrated = 0
        
        for migration_func in migrations:
            try:
                count = migration_func()
                total_migrated += count
                time.sleep(1)  # Small delay between migrations
            except Exception as e:
                print(f"⚠️ Migration step failed: {e}")
        
        print(f"\n✅ TOTAL MIGRATED: {total_migrated} records")
        self.print_stats()
    
    def migrate_candidates(self):
        """Migrate candidates data"""
        print("\n👤 Migrating candidates...")
        
        # Fetch from Supabase
        response = self.supabase.table("candidates").select("*").execute()
        candidates = response.data
        
        if not candidates:
            print("   No candidates found in Supabase")
            return 0
        
        migrated = 0
        for candidate in candidates:
            mongo_candidate = {
                "name": candidate.get("name", ""),
                "email": candidate.get("email", ""),
                "phone": candidate.get("phone", ""),
                "experience": candidate.get("experience", 0),
                "skills": candidate.get("skills", []),
                "created_at": candidate.get("created_at", datetime.now()),
                "updated_at": datetime.now(),
                "metadata": {
                    "supabase_id": candidate.get("id"),
                    "migrated_at": datetime.now()
                }
            }
            
            # Insert into MongoDB
            self.mongo_db.candidates.insert_one(mongo_candidate)
            migrated += 1
        
        print(f"   ✅ Migrated {migrated} candidates")
        return migrated
    
    def migrate_jobs(self):
        """Migrate jobs data"""
        print("\n💼 Migrating jobs...")
        
        response = self.supabase.table("jobs").select("*").execute()
        jobs = response.data
        
        if not jobs:
            print("   No jobs found in Supabase")
            return 0
        
        migrated = 0
        for job in jobs:
            mongo_job = {
                "title": job.get("title", ""),
                "description": job.get("description", ""),
                "requirements": job.get("requirements", []),
                "status": job.get("status", "open"),
                "department": job.get("department", ""),
                "location": job.get("location", ""),
                "salary_range": job.get("salary_range", {}),
                "created_at": job.get("created_at", datetime.now()),
                "updated_at": datetime.now(),
                "metadata": {
                    "supabase_id": job.get("id"),
                    "migrated_at": datetime.now()
                }
            }
            
            self.mongo_db.jobs.insert_one(mongo_job)
            migrated += 1
        
        print(f"   ✅ Migrated {migrated} jobs")
        return migrated
    
    def migrate_applications(self):
        """Migrate applications data"""
        print("\n📄 Migrating applications...")
        
        response = self.supabase.table("applications").select("*").execute()
        applications = response.data
        
        if not applications:
            print("   No applications found in Supabase")
            return 0
        
        migrated = 0
        for app in applications:
            mongo_app = {
                "candidate_id": app.get("candidate_id"),
                "job_id": app.get("job_id"),
                "resume_url": app.get("resume_url", ""),
                "resume_status": app.get("resume_status", "open"),
                "rag_uploaded": app.get("rag_uploaded", False),
                "cover_letter": app.get("cover_letter", ""),
                "applied_at": app.get("created_at", datetime.now()),
                "created_at": datetime.now(),
                "updated_at": datetime.now(),
                "metadata": {
                    "supabase_id": app.get("id"),
                    "migrated_at": datetime.now()
                }
            }
            
            self.mongo_db.applications.insert_one(mongo_app)
            migrated += 1
        
        print(f"   ✅ Migrated {migrated} applications")
        return migrated
    
    def print_stats(self):
        """Print migration statistics"""
        print("\n" + "="*60)
        print("📊 POST-MIGRATION STATISTICS")
        print("="*60)
        
        stats = {
            "Candidates": self.mongo_db.candidates.count_documents({}),
            "Jobs": self.mongo_db.jobs.count_documents({}),
            "Applications": self.mongo_db.applications.count_documents({}),
            "Open Applications": self.mongo_db.applications.count_documents({"resume_status": "open"}),
            "Indexed Applications": self.mongo_db.applications.count_documents({"rag_uploaded": True}),
        }
        
        for key, value in stats.items():
            print(f"{key}: {value}")
        
        print("="*60)

if __name__ == "__main__":
    migrator = DataMigrator()
    migrator.migrate_all_data()

🚀 Data Migration Tool Initialized

📦 STARTING DATA MIGRATION: Supabase → MongoDB Atlas

👤 Migrating candidates...
   ✅ Migrated 69 candidates

💼 Migrating jobs...
   ✅ Migrated 5 jobs

📄 Migrating applications...
   ✅ Migrated 70 applications

✅ TOTAL MIGRATED: 144 records

📊 POST-MIGRATION STATISTICS
Candidates: 69
Jobs: 5
Applications: 70
Open Applications: 0
Indexed Applications: 70
